In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPool2D
from keras.models import Model
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import weave

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
keras.backend.image_data_format()

'channels_first'

In [3]:
def lr_schedule(epoch):
    return (0.00100 - 7e-6*epoch)

def straight_stack(x, num_filters, rep = 3, weight_decay = 1e-4, dropout_start = 0.2, dropout_add = .1):
    for n in range(rep):
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = MaxPool2D()(x)
        x = Dropout(dropout_start+n*dropout_add)(x)
    
    return x

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape)
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

batch_size = 64

(50000, 3, 32, 32)


In [5]:
viewer = keras.callbacks.TensorBoard(log_dir='./logs', 
                                     histogram_freq=0, 
                                     batch_size=batch_size, 
                                     write_graph=True)
num_filters = 4
num_val_test = 500
epochs = 1
steps_per_epoch = x_train.shape[0] // batch_size
model_details = {}
conv_layers = 3
for num_stack in [4,2,1]:
    inputs = Input(shape=(3,32,32))
    input_array = []
    pyrm_layers = int(np.log2(num_stack))
    for stack in range(num_stack):
        x = straight_stack(inputs, num_filters/num_stack, rep = conv_layers - pyrm_layers)
        input_array.append(x)
    if pyrm_layers > 0:
        input_size = tuple(input_array[0].shape[1:].as_list())
        print(input_size)
        x = weave.pyrm_net(inputs = input_array,
                           n_layers = pyrm_layers,
                           n_filters_start = input_size[0]*2,
                           n_gpus = 1,
                           r_filter = 4,
                           r_combine= 2,
                           min_dim = 2,
                           max_pool_loc = 1,
                           pure_combine = False)
        x = BatchNormalization()(x)
        x = MaxPool2D()(x)
    else:
        pass
    print("output shape of conv layers: ",x.shape)
    x = Flatten()(x)
    
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=[inputs], outputs=predictions)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary()
    num_params = model.count_params()
    print(num_params)
    print('number of parameters %d' % num_params)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch= steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                    callbacks=[LearningRateScheduler(lr_schedule), viewer])
    model_details[num_stack] = (num_params, model.evaluate(x_test,y_test))
    print(model_details)
    
    
    

(1, 16, 16)
GPU settings allow for inf layers
Minimum output size allow for 6 layers
Number of layers 2
First Layer Size: 2 (Number of Units)
First Layer Output Size: 2 (Number of Tensors)
Final Layer Size 1
('output shape of conv layers: ', TensorShape([Dimension(None), Dimension(16), Dimension(4), Dimension(4)]))
5850
number of parameters 5850
Epoch 1/1
10000/10000 [==============================] - 44s 4ms/step
{4: (5850, [1.6624127925872803, 0.3934])}
(4, 8, 8)
GPU settings allow for inf layers
Minimum output size allow for 4 layers
Number of layers 1
First Layer Size: 1 (Number of Units)
First Layer Output Size: 1 (Number of Tensors)
Final Layer Size 1
('output shape of conv layers: ', TensorShape([Dimension(None), Dimension(16), Dimension(4), Dimension(4)]))
5798
number of parameters 5798
Epoch 1/1
10000/10000 [==============================] - 34s 3ms/step
{2: (5798, [1.6725436157226563, 0.3969]), 4: (5850, [1.6624127925872803, 0.3934])}
('output shape of conv layers: ', TensorS

In [6]:
model_details


{1: (7646, [1.6588449335098268, 0.3939]),
 2: (5798, [1.6725436157226563, 0.3969]),
 4: (5850, [1.6624127925872803, 0.3934])}

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 3, 32, 32)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 4, 32, 32)         112       
_________________________________________________________________
batch_normalization_25 (Batc (None, 4, 32, 32)         128       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 4, 32, 32)         148       
_________________________________________________________________
batch_normalization_26 (Batc (None, 4, 32, 32)         128       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 4, 16, 16)         0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 4, 16, 16)         0         
__________

In [8]:
num_filters = 32
num_val_test = 500
epochs = 65
model_details = {}
conv_layers = 3
num_stack = 4
inputs = Input(shape=(3,32,32))
input_array = []
pyrm_layers = 2
for stack in range(num_stack):
    x = straight_stack(inputs, num_filters/num_stack, rep = 1)
    input_array.append(x)
input_size = tuple(input_array[0].shape[1:].as_list())
print(input_size)
x = weave.pyrm_net(input_size=input_size,
                   n_layers = pyrm_layers,
                   n_filters_start = input_size[0]*2,
                   n_gpus = 1,
                   inputs = input_array,
                   r_filter = 2,
                   r_combine= 1,
                   min_dim = 2,
                   max_pool_loc = 2,
                   pure_combine = True,
                   end_max_pool = False)

print("output shape of conv layers: ",x.shape)
x = Flatten()(x)

predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


(8, 16, 16)


TypeError: pyrm_net() got an unexpected keyword argument 'input_size'

In [ ]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                steps_per_epoch=x_train.shape[0] // batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                callbacks=[LearningRateScheduler(lr_schedule)])
model.evaluate(x_test,y_test)

In [ ]:
model.evaluate(x_test,y_test)